In [9]:
from fredapi import Fred
import pandas as pd

In [10]:
from fredapi import Fred

fred = Fred(api_key="91349b4b1b532715d85f4513d8188365")
tenors = ["DGS1", "DGS2", "DGS5", "DGS10", "DGS30"]

# fetch each separately and concat
frames = []
for s in tenors:
    ts = fred.get_series(s, observation_start="2005-01-01")
    ts.name = s
    frames.append(ts)

df = pd.concat(frames, axis=1).ffill().dropna()
df.columns = ["Y1","Y2","Y5","Y10","Y30"]

In [12]:
df['S2_10'] = df['Y10'] - df['Y2']
df['S5_30'] = df['Y30'] - df['Y5']

In [13]:
window = 60
df['S2_10_mean'] = df['S2_10'].rolling(window).mean()
df['S2_10_std']  = df['S2_10'].rolling(window).std()
df['S2_10_z']    = (df['S2_10'] - df['S2_10_mean']) / df['S2_10_std']

In [14]:
df['Y10_vol'] = df['Y10'].ewm(halflife=21).std()

In [15]:
df['S2_10_next']    = df['S2_10'].shift(-21)
df['Carry_2_10']    = df['S2_10_next'] - df['S2_10']